## GLORI alignment and pileup pipeline

Jianheng Liu (Fox) @ Jaffrey Lab, May 31st, 2023

Concat: jil4026@med.cornell.edu

**Usage:** Modify `Prepartion` section to fit your computer. Modify `Variable` section before each run.All outputs will be saved in the `workpath`. Logs will be saved in the notebook. Don't close the `Notebook` before it finished (keep the backend at least)!

**Note 1:** Compressed `tmp` file and `pileup.txt` file can work with the latest scripts.

**Note 2:** This panel notebook should be compatible with `Python 3`. But please use `Python 2` to run the analytic scripts.

In [1]:
# local time
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
print("Started:", current_time)

# Show notebook directory
!pwd -P

Started: 05/31/23 13:52:34
/home/fox/Jupyter/GLORI_Yi_lab/Jupyter_notebook


### 0. Variable here

In [2]:
name = "test2" # file name prefix
folder = "test2_run/"
path = "./" # output path: workpath=path/folder
workpath = path + "/" + folder + "/"
read1 = "test2.fastq.gz" # fastq or fastq.gz
pileup_processors = 10

### 1. Preparation

**Files** (some of them are prefix)

In [3]:
ref_genome_index = "/home/fox/Database/GLORI_index/GRCh38_r104/hisat2_index/"
ref_genome = "/home/fox/Database/GLORI_index/GRCh38_r104/hisat2_index/Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa"
db = "/home/fox/Database/Genome/Ensembl/GRCh38_r104/Homo_sapiens.GRCh38.104.noheader.base.sorted.db"

**Software** (For python, use the fixed version)

In [4]:
python = '/home/fox/Software/bin/python'
bowtie2 = '/home/fox/Software/bin/python2'
samtools = '/home/fox/Software/samtools/1.16/bin/samtools'
cutadapt = '/home/fox/Software/bin/cutadapt'
hisat2_path = '/home/fox/Software/hisat2/2.1.0/'
umitools = '/home/fox/Software/bin/umi_tools'

**Scripts**

In [5]:
hisat2_script = "../A2G_hisat2.py"
pileup_script = "../pileup_genome.py"
formater_script = "../format_pileups.py" 

**Create the workpath if not exist, then move to the workpath**

In [6]:
import os
if os.path.isdir(workpath) == False:
    os.mkdir(workpath)
os.chdir(workpath)

### 2. QC

In [7]:
# Extract UMI
UMI_out = "{name}.UMI.fastq".format(name=name)
UMI_log = "{name}.UMI.log".format(name=name)

!$umitools extract -I ../$read1 -S $UMI_out -p NNNNNNNNNNNN --log=$UMI_log

In [8]:
adapter = "AGATCGGAAGAGCGTCGTG"
cutadapt_out = "{name}.cutadapt.fastq".format(name=name)

!$cutadapt -a $adapter --max-n 0 --trimmed-only -e 0.1 -q 30 -m 30 --trim-n -o $cutadapt_out $UMI_out

This is cutadapt 4.1 with Python 3.9.14
Command line parameters: -a AGATCGGAAGAGCGTCGTG --max-n 0 --trimmed-only -e 0.1 -q 30 -m 30 --trim-n -o test2.cutadapt.fastq test2.UMI.fastq
Processing single-end reads on 1 core ...
Done           00:00:20     1,000,000 reads @  20.4 µs/read;   2.94 M reads/minute
Finished in 20.45 s (20 µs/read; 2.93 M reads/minute).

=== Summary ===

Total reads processed:               1,000,000
Reads with adapters:                   945,794 (94.6%)

== Read fate breakdown ==
Reads that were too short:             187,739 (18.8%)
Reads with too many N:                      95 (0.0%)
Reads discarded as untrimmed:           15,868 (1.6%)
Reads written (passing filters):       796,298 (79.6%)

Total basepairs processed:   138,000,000 bp
Quality-trimmed:              34,733,029 bp (25.2%)
Total written (filtered):     38,591,616 bp (28.0%)

=== Adapter 1 ===

Sequence: AGATCGGAAGAGCGTCGTG; Type: regular 3'; Length: 19; Trimmed: 945794 times

Minimum overlap: 3
No

### 2. Mapping

In [9]:
!$python $hisat2_script -F $cutadapt_out -o hisat2_genome -I $ref_genome_index --index-prefix HISAT2 --hisat2-path $hisat2_path --del-convert --del-sam

[2023-05-31 13:53:17]Converting test2.cutadapt.fastq, A2G...
[2023-05-31 13:53:19]Mapping with hisat2, TEMP prefix: hisat2_1656473
[2023-05-31 13:53:45]T2C report:
[2023-05-31 13:53:45]A2G report:
[2023-05-31 13:53:45]Handling SAM outputs...
[2023-05-31 13:54:18]Completed successfully:
 Total reads: 796298
 Unique mapping: 372471 (46.775%)
   A2G: 185192 (23.26%)
   T2C: 187279 (23.52%)
 Multiple mapping: 109516 (13.753%)
 Unmapped: 314311 (39.472%)


In [10]:
!$samtools sort -@ 4 -m 4G -o hisat2_genome.sorted.bam hisat2_genome.bam

[bam_sort_core] merging from 0 files and 4 in-memory blocks...


In [11]:
!$samtools index hisat2_genome.sorted.bam

In [12]:
!$umitools dedup --stdin=hisat2_genome.sorted.bam --log=umi.logs --method=unique > hisat2_genome.sorted.umi.bam

In [13]:
!samtools sort -@ 4 -m 4G -o hisat2_genome.sorted.umi.sorted.bam  hisat2_genome.sorted.umi.bam

[bam_sort_core] merging from 0 files and 4 in-memory blocks...


In [14]:
!samtools index hisat2_genome.sorted.umi.sorted.bam

### 3. Pileup

In [15]:
pileup_output = "{}.pileups.tmp".format(name)
!$python $pileup_script -P $pileup_processors -i hisat2_genome.sorted.umi.sorted.bam -o $pileup_output -f $ref_genome

[2023-05-31 13:54:42] Pileup genome, processers: 10, pid: 1656781
[2023-05-31 13:57:40] Merging TEMPs
[2023-05-31 13:57:45] Genome pileup finished.


### 4. Format output

In [16]:
format_output = "{}.pileups.txt".format(name)
format_output_CR = "{}.pileups.CR".format(name)

!$python $formater_script -i $pileup_output -o $format_output  --CR $format_output_CR --db $db

[2023-05-31 13:57:45] Reading input...
[2023-05-31 13:57:54] 1000000 items processed...
[2023-05-31 13:58:01] 2000000 items processed...
[2023-05-31 13:58:08] 3000000 items processed...
[2023-05-31 13:58:14] 4000000 items processed...
[2023-05-31 13:58:21] 5000000 items processed...
[2023-05-31 13:58:28] 6000000 items processed...
[2023-05-31 13:58:35] 7000000 items processed...
[2023-05-31 13:58:41] 8000000 items processed...
[2023-05-31 13:58:49] 9000000 items processed...
[2023-05-31 13:58:55] 10000000 items processed...
[2023-05-31 13:59:02] 11000000 items processed...
[2023-05-31 13:59:10] 12000000 items processed...
[2023-05-31 13:59:17] 13000000 items processed...
[2023-05-31 13:59:25] 14000000 items processed...
[2023-05-31 13:59:31] 15000000 items processed...
[2023-05-31 13:59:37] 16000000 items processed...
[2023-05-31 13:59:43] 17000000 items processed...
[2023-05-31 13:59:50] 18000000 items processed...
[2023-05-31 13:59:58] 19000000 items processed...
[2023-05-31 14:00:04

Output file: test2.pileups.txt

### 5. Clean up to save disk (optional)

### 6. When and where am I

In [17]:
!pwd -P
!ls -l

/home/fox/Jupyter/GLORI_Yi_lab/Jupyter_notebook/test2_run
total 1309548
-rw-rw-r-- 1 fox fox  22847327 May 31 13:54 hisat2_genome.bam
-rw-rw-r-- 1 fox fox  15010055 May 31 13:54 hisat2_genome.multimappers.bam
-rw-rw-r-- 1 fox fox  20006009 May 31 13:54 hisat2_genome.sorted.bam
-rw-rw-r-- 1 fox fox   2045912 May 31 13:54 hisat2_genome.sorted.bam.bai
-rw-rw-r-- 1 fox fox  19309616 May 31 13:54 hisat2_genome.sorted.umi.bam
-rw-rw-r-- 1 fox fox  19309641 May 31 13:54 hisat2_genome.sorted.umi.sorted.bam
-rw-rw-r-- 1 fox fox   2046416 May 31 13:54 hisat2_genome.sorted.umi.sorted.bam.bai
-rw-rw-r-- 1 fox fox 123062987 May 31 13:53 test2.cutadapt.fastq
-rw-rw-r-- 1 fox fox  66267666 May 31 13:54 test2.cutadapt.unmapped.fastq
-rw-rw-r-- 1 fox fox    580151 May 31 14:19 test2.pileups.CR
-rw-rw-r-- 1 fox fox 361417014 May 31 13:57 test2.pileups.tmp
-rw-rw-r-- 1 fox fox 355383290 May 31 14:19 test2.pileups.txt
-rw-rw-r-- 1 fox fox 333612354 May 31 13:52 test2.UMI.fastq
-rw-rw-r-- 1 fox fox     197

In [18]:
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
print("Finished:", current_time)

Finished: 05/31/23 14:19:38


## Don't forget to save the Notebook.